In [ ]:
### Download buildings and streets from OSM ###

In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# set coords in epsg:4326 (WGS84) and radius in meters
# coords = (41.146099, -8.610943) # Porto, metris crs - EPSG:3763
# coords = (38.713622, -9.139358) # Lisbon, metris crs - EPSG:3763
# coords = (37.787889, -122.407698) # San Francisco, metris crs - EPSG:7131
coords = (22.326565, 114.167448) # Hong Kong, metris crs - EPSG:2326
radius = 2000

In [ ]:
# create bounding box from coordinates and radius
box = ox.utils_geo.bbox_from_point(coords, radius)

In [ ]:
# set path variables
folder = r'C:/folder'
# export to these files according to geometry type
path_0 = folder + '/HK_buildings.csv' 
path_1 = folder + '/HK_streets_lines.csv'
path_2 = folder + '/HK_streets_polygons.csv'

In [ ]:
# download buildings with double the radius
buildings = ox.geometries.geometries_from_bbox(box[0],
                                               box[1],
                                               box[2],
                                               box[3],
                                               tags={'building': True})                                          
print(buildings.head())

In [ ]:
# clean irrelevant fields from buildings
buildings.dropna(axis = 1, inplace = True)

# remove point geometry entries
for row in buildings.itertuples(name = 'road'):
    if type(row[1]) == Point:
        buildings.drop(axis = 0, index = row[0], inplace = True)
print(buildings.head())
print(buildings.shape)

In [ ]:
# re-project buildings to the proper metric crs
buildings.to_crs('epsg:2326', inplace = True)
buildings.head()

In [ ]:
# export csv
buildings.to_csv(path_0, sep = '~', na_rep = 'NaN', header = False, index = False, encoding = 'utf-8')

In [ ]:
# download street network with double the radius for better accuracy of centrality calculations
streets = ox.geometries.geometries_from_bbox(box[0],
                                             box[1],
                                             box[2],
                                             box[3],
                                             tags={'highway': True})

In [ ]:
# clean irrelevant fields from streets
streets = streets.dropna(axis = 1)
streets.head()

In [ ]:
# list for linestring entries
streets_l_list = []
# remove point and polygon geometry entries
# the index of the geometry can change according to source layer - it is usually 1 or 2
for row in streets.itertuples(name = 'road'):
    if type(row[1]) == LineString:
        streets_l_list.append(row)

In [ ]:
# order of columns might change depending on source layer
streets_l = gpd.GeoDataFrame.from_records(streets_l_list, columns = ['osmid', 'geometry', 'highway'])
streets_l.set_crs('epsg:4326', inplace = True)

In [ ]:
# re-project streets to the proper metric crs
streets_l.to_crs('epsg:2326', inplace = True)
streets_l = streets_l[['geometry', 'highway']]

In [ ]:
# export csv linestring
streets_l.to_csv(path_1, sep = '~', na_rep = 'NaN', header = False, index = False, encoding = 'utf-8')

In [ ]:
# list for polygon entries
streets_p_list = []
# remove point and linestring geometry entries
# the index of the geometry can change according to source layer - it is usually 1 or 2
for row in streets.itertuples(name = 'road'):
    if type(row[1]) == Polygon:
        streets_p_list.append(row)

In [ ]:
# order of columns might change depending on source layer
streets_p = gpd.GeoDataFrame.from_records(streets_p_list, columns = ['osmid', 'geometry','highway'])
streets_p.set_crs('epsg:4326', inplace = True)

In [ ]:
# re-project streets to the proper metric crs
streets_p.to_crs('epsg:2326', inplace = True)
streets_p = streets_p[['geometry', 'highway']]

In [ ]:
# export csv polygon
streets_p.to_csv(path_2, sep = '~', na_rep = 'NaN', header = False, index = False, encoding = 'utf-8')